In [10]:
import sys
sys.path.append('..')
from src.common import *
sys.path.append('../questions_construction')
from src.questions_construction.main import QUESTION_CATEGORIES

from collections import defaultdict

In [11]:
domain_name = 'blocksworld'
instance_name = 'Instance_1'

model_name = 'gemini'
eval_type = 'zero_shot'

results_path = f'{RESULTS_PATH}/{model_name}/{eval_type}/{domain_name}/{instance_name}.jsonl'
results = open_jsonl(results_path)

In [12]:
by_question_category = {v: defaultdict(list) for v in QUESTION_CATEGORIES}

for d in results:
    by_question_category[d[OUT_OBJ_QUESTION_CATEGORY]][d[OUT_OBJ_ANSWER_TYPE]].append(d)

In [ ]:
import 